# Gaia TGAS + 2MASS + WISE

The provided Gaia dataset is a dump of the Gaia science archive's match between the astrometry in the Tycho-Gaia Astrometric Solution (TGAS) and photometric sources in 2MASS and WISE. The data contains all of the columns provided in the TGAS catalog ([described here](https://www.cosmos.esa.int/web/gaia/dr1)) along with photometric columns from the 2MASS and WISE catalogs. 

The 2MASS photometry is in the $J$, $H$, and $K_s$ bands, so the corresponding magnitude measurements are stored in the columns `j_m`, `h_m`, `ks_m`. The uncertainties for each magnitude balue are in the columns `j_msigcom`, `h_msigcom`, `ks_msigcom`.

The WISE filters are named $W1$-$W4$, so the magnitudes and uncertainties are `w*mpro` and `w*mpro_error` with the `*` replaced by the filter number, 1-4.

In [ ]:
from os import path
import numpy as np

import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
plt.style.use('notebook.mplstyle')
%matplotlib inline
import numpy as np

In [ ]:
data_path = '../data/'

# Reading the Gaia TGAS data

The data are all stored in a single FITS binary table and can be read using the `astropy.table.Table` class:

In [ ]:
tgas = Table.read(path.join(data_path, 'gaia', 'tgas_2mass_wise.fits'))

In [ ]:
print(tgas.colnames)

---

As an example of using the data, let's make two color-magnitude diagrams for all stars within 150 pc:

In [ ]:
with u.set_enabled_equivalencies(u.parallax()):
    dist = coord.Distance((tgas['parallax'] * u.mas).to(u.pc), 
                          allow_negative=True)
    
dist_cut = (dist < 150. * u.pc) & (dist > 0)

Compute absolute magnitude using the distance to each star, and compute colors by differencing magnitudes:

In [ ]:
M_G = tgas['phot_g_mean_mag'][dist_cut] - dist[dist_cut].distmod.value
G_J = tgas['phot_g_mean_mag'][dist_cut] - tgas[dist_cut]['j_m']
G_W1 = tgas['phot_g_mean_mag'][dist_cut] - tgas[dist_cut]['w1mpro']

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
axes[0].plot(G_J, M_G, marker=',', linestyle='none')
axes[1].plot(G_W1, M_G, marker=',', linestyle='none')

axes[0].set_xlim(-0.25, 2.5)
axes[0].set_ylim(9, -1)

axes[1].set_xlim(-0.25, 3.5)

axes[0].set_xlabel('$G-J$ [mag]')
axes[1].set_xlabel('$G-W1$ [mag]')
axes[0].set_ylabel('$M_G$ [mag]')